# Practice training a deep neural network on the CIFAR10 image dataset:

In [1]:
import os
os.chdir("..")

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

from PIL import Image
import time

a. Build a DNN with 20 hidden layers of 100 neurons each (that’s too many, but it’s the point of this exercise). Use He initialization and the Swish activation function.

In [4]:
batch_size = 4

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0, 0, 0), (1, 1, 1))]
)

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
def weights_init(layer):
    if isinstance(layer, nn.Linear):
        torch.nn.init.kaiming_normal_(layer.weight, mode='fan_in')
        # torch.nn.init.zero_(m.bias)

In [11]:
layers = [nn.Linear(100, 100) for layer in range(18)]

CIFAR10 = nn.Sequential(
    nn.Linear(3*32*32, 100),
    *layers,
    nn.Linear(100, 10),
    nn.Softmax(dim=-1)
).apply(weights_init)

In [12]:
epochs = 2
def train(model, trainloader, epochs, crtireion, learning_rate=5e-1):
    running_loss = 0.0

    model.train()
    optimizer = optim.NAdam(params=CIFAR10.parameters(), lr=learning_rate)
    flatten = nn.Flatten()
    for epoch in range(epochs):
        for i, data in enumerate(trainloader):
            inputs, targets = data
    
            optimizer.zero_grad()
            inputs = flatten(inputs)
            outputs = model(inputs)
            loss = crtireion(outputs, targets)
            loss.backward()
            optimizer.step()
            
            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
                running_loss = 0.0

In [14]:
crtireion = nn.CrossEntropyLoss()

start = time.time()
train(CIFAR10, trainloader, epochs=2, crtireion=crtireion)
end = time.time()
print(f"training took {end-start}s")

[1,  2000] loss: 2.361
[1,  4000] loss: 2.361
[1,  6000] loss: 2.358
[1,  8000] loss: 2.362
[1, 10000] loss: 2.365
[1, 12000] loss: 2.361
[2,  2000] loss: 2.955
[2,  4000] loss: 2.356
[2,  6000] loss: 2.362
[2,  8000] loss: 2.363
[2, 10000] loss: 2.364
[2, 12000] loss: 2.358
training took 445.9984600543976s


In [32]:
def evaluate(model, testloader):
    correct = 0.0
    total = 0.0
    
    model.eval()
    flatten = nn.Flatten()
    for i, data in enumerate(testloader):
        test_inputs, test_targets = data
        test_inputs = flatten(test_inputs)
        test_outputs = model(test_inputs)

        _, predictions = torch.max(test_outputs, 1)

        total += test_targets.size(0)
        correct += (predictions == test_targets).sum().item()

    return total, correct

In [33]:
total, correct = evaluate(CIFAR10, testloader)

In [34]:
print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 10.0 %
